# Assignment 4 – CSCN8030

# Adding imports

In [4]:
import pandas as pd

# Load the dataset

In [6]:
df = pd.read_excel("./data/loblaws.xlsx")
df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Data,Variable,Discerption
1,NaN,E Comm,CustomerID,Unique customer ID
2,NaN,E Comm,Churn,Churn Flag
3,NaN,E Comm,Tenure,Tenure of customer in organization
4,NaN,E Comm,PreferredLoginDevice,Preferred login device of customer
